In [410]:
!pip install -q sentence-transformers torch numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [411]:
import pandas as pd

test = pd.read_csv('test.csv', index_col=False)
data = pd.read_csv('dataset.csv', index_col=False)
categories = pd.read_csv('categories.csv',names=["Category"])

In [412]:
test.head()

,Title
0,What It Takes: Lessons in the Pursuit of Excel...
1,The Practice: Shipping Creative Work by Seth G...
2,The Ideal Team Player: How to Recognize and Cu...
3,4 Essential Keys to Effective Communication in...
4,The Undocumented Americans by Karla Cornejo Vi...


In [413]:
categories.head()

,Category
0,Books on Psychology
1,Books on Leadership
2,Books on Investing
3,Books on Sales
4,Books on Finance


In [414]:
data.head()

,Title,Categories
0,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...
1,Eat That Frog! 21 Ways to Stop Procrastinating...,Books on Procrastination\nBooks on Time Manage...
2,12 Rules For Life: An Antidote to Chaos by Jor...,Books on Psychology\nBooks on Self-Help\nBooks...
3,Deep Work: Rules for Focused Success in a Dist...,Books on Productivity\nBooks on Focus\nBooks o...
4,Dare to Lead: Brave Work. Tough Conversations....,Books on Leadership\nBooks on Personal Develop...


In [415]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, CrossEncoder, util, losses
from tqdm import tqdm 

In [416]:
df = pd.read_csv('dataset.csv')

In [417]:
df.head()

,Title,Categories
0,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...
1,Eat That Frog! 21 Ways to Stop Procrastinating...,Books on Procrastination\nBooks on Time Manage...
2,12 Rules For Life: An Antidote to Chaos by Jor...,Books on Psychology\nBooks on Self-Help\nBooks...
3,Deep Work: Rules for Focused Success in a Dist...,Books on Productivity\nBooks on Focus\nBooks o...
4,Dare to Lead: Brave Work. Tough Conversations....,Books on Leadership\nBooks on Personal Develop...


In [418]:
df['CategoriesList'] = df['Categories'].str.split('\n')

In [332]:
len(df)

1217

In [333]:
df_exploded = df.explode('CategoriesList').reset_index(drop=True)

In [334]:
df_exploded.head(20)

,Title,Categories,CategoriesList
0,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...,Books on Organization
1,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...,Books on Minimalism
2,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...,Books on Japanese Art
3,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...,Books on Self-Help
4,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...,Books on Decluttering
5,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...,Books on Home Improvement
6,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...,Books on Personal Development
7,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...,Books on Cleaning
8,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...,Books on Productivity
9,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization\nBooks on Minimalism\nBo...,Books on Lifestyle


In [351]:
import pandas as pd

# 0) assume you've already loaded:
#    df:         columns = ['Title','CategoriesList']
# If CategoriesList is still a string, first do:
# df['CategoriesList'] = df['Categories'].str.split('\n')

records = []
for _, row in df.iterrows():
    anchor = row['Title']
    cats   = row['CategoriesList']
    # skip rows where CategoriesList isn't a list
    if not isinstance(cats, list):
        continue
    
    # emit one record per category in this list
    for pos in cats:
        records.append({
            'anchor':   anchor,
            'positive': pos
        })

# build the final 2-column DataFrame
df_pairs = pd.DataFrame(records, columns=['anchor','positive'])

print(df_pairs.shape)
print(df_pairs.head(10))


(2722, 2)
                                              anchor  \
0  The Life-Changing Magic of Tidying Up: The Jap...   
1  The Life-Changing Magic of Tidying Up: The Jap...   
2  The Life-Changing Magic of Tidying Up: The Jap...   
3  The Life-Changing Magic of Tidying Up: The Jap...   
4  The Life-Changing Magic of Tidying Up: The Jap...   
5  The Life-Changing Magic of Tidying Up: The Jap...   
6  The Life-Changing Magic of Tidying Up: The Jap...   
7  The Life-Changing Magic of Tidying Up: The Jap...   
8  The Life-Changing Magic of Tidying Up: The Jap...   
9  The Life-Changing Magic of Tidying Up: The Jap...   

                        positive  
0          Books on Organization  
1            Books on Minimalism  
2          Books on Japanese Art  
3             Books on Self-Help  
4          Books on Decluttering  
5      Books on Home Improvement  
6  Books on Personal Development  
7              Books on Cleaning  
8          Books on Productivity  
9             Books on 

In [352]:
df_pairs

,anchor,positive
0,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization
1,The Life-Changing Magic of Tidying Up: The Jap...,Books on Minimalism
2,The Life-Changing Magic of Tidying Up: The Jap...,Books on Japanese Art
3,The Life-Changing Magic of Tidying Up: The Jap...,Books on Self-Help
4,The Life-Changing Magic of Tidying Up: The Jap...,Books on Decluttering
...,...,...
2717,The Rise of Superman: Decoding the Science of ...,Books on Human Behavior
2718,The Rise of Superman: Decoding the Science of ...,Books on Positive Thinking
2719,Men Explain Things To Me by Rebecca Solnit,"Books on Feminism, Books on Gender Studies, Bo..."
2720,"The Science of Self-Discipline: The Willpower,...","Books on Self-Discipline, Books on Mental Toug..."


In [353]:
len(df_pairs)

2722

In [354]:
df_pairs = df_pairs.dropna(subset=["anchor", "positive"])
df_pairs["anchor"]    = df_pairs["anchor"].astype(str)
df_pairs["positive"] = df_pairs["positive"].astype(str)

df_pairs.to_csv("df_pairs.csv", index=False)

In [355]:
len(df_pairs)

2722

In [228]:
df_train = df_exploded[['Title', 'CategoriesList']].rename(columns={'CategoriesList': 'Category'})

In [229]:
df_train.head()

,Title,Category
0,The Life-Changing Magic of Tidying Up: The Jap...,Books on Organization
1,The Life-Changing Magic of Tidying Up: The Jap...,Books on Minimalism
2,The Life-Changing Magic of Tidying Up: The Jap...,Books on Japanese Art
3,The Life-Changing Magic of Tidying Up: The Jap...,Books on Self-Help
4,The Life-Changing Magic of Tidying Up: The Jap...,Books on Decluttering


In [230]:
df_train_cleaned = df_train.copy()

In [231]:
df_train_cleaned['Category'] = df_train_cleaned['Category'].str.replace('Books on', '')

In [232]:
df_train_cleaned.head()

,Title,Category
0,The Life-Changing Magic of Tidying Up: The Jap...,Organization
1,The Life-Changing Magic of Tidying Up: The Jap...,Minimalism
2,The Life-Changing Magic of Tidying Up: The Jap...,Japanese Art
3,The Life-Changing Magic of Tidying Up: The Jap...,Self-Help
4,The Life-Changing Magic of Tidying Up: The Jap...,Decluttering


In [233]:
df_train_cleaned = df_train_cleaned.dropna(subset=["Title", "Category"])

In [234]:
df_train_cleaned["Title"]    = df_train_cleaned["Title"].astype(str)
df_train_cleaned["Category"] = df_train_cleaned["Category"].astype(str)


In [235]:
len(df_train_cleaned)

2722

In [249]:
df_train_cleaned.to_csv("df_train_cleaned.csv", index=False)

In [245]:
df_train_cleaned.head()

,Title,Category,label
0,The Life-Changing Magic of Tidying Up: The Jap...,Organization,1
1,The Life-Changing Magic of Tidying Up: The Jap...,Minimalism,1
2,The Life-Changing Magic of Tidying Up: The Jap...,Japanese Art,1
3,The Life-Changing Magic of Tidying Up: The Jap...,Self-Help,1
4,The Life-Changing Magic of Tidying Up: The Jap...,Decluttering,1


In [242]:
df_train_cleaned['label'] = 1

In [247]:
df_train_cleaned = df_train_cleaned.rename(columns={"Title":"text1", "Category":"text2"})

In [248]:
df_train_cleaned.head()

,text1,text2,label
0,The Life-Changing Magic of Tidying Up: The Jap...,Organization,1
1,The Life-Changing Magic of Tidying Up: The Jap...,Minimalism,1
2,The Life-Changing Magic of Tidying Up: The Jap...,Japanese Art,1
3,The Life-Changing Magic of Tidying Up: The Jap...,Self-Help,1
4,The Life-Changing Magic of Tidying Up: The Jap...,Decluttering,1


In [187]:
from sentence_transformers import InputExample

examples = [
    InputExample(texts=[row["Title"], row["Category"]])
    for _, row in df_train_cleaned.iterrows()
]


In [190]:
train_dataloader = DataLoader(
    examples,
    batch_size=32,
    shuffle=True,
    collate_fn=lambda batch: batch
)

model = SentenceTransformer('all-mpnet-base-v2')
train_loss = losses.MultipleNegativesRankingLoss(model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    report_to="none"
)


TypeError: FitMixin.fit() got an unexpected keyword argument 'report_to'

In [184]:
from datasets import Dataset, Features, Value

from torch.utils.data import Dataset, DataLoader
from sentence_transformers import InputExample

class BookCategoryDataset(Dataset):
    def __init__(self, df):
        # reset_index so iloc[0..len-1] works
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]          
        return InputExample(texts=[row['Title'], row['Category']])

# now:
dataset = BookCategoryDataset(df_train_cleaned)
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=lambda batch: batch)


model = SentenceTransformer('all-mpnet-base-v2')
train_loss = losses.TripletLoss(model=model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100
)

ArrowTypeError: Expected bytes, got a 'float' object

In [356]:
from datasets import load_dataset

# 1. Load your CSV; this gives a DatasetDict with a single "train" split
dataset = load_dataset("csv", data_files="df_pairs.csv")

# 2. Shuffle and split off 10% for validation
split = dataset["train"].shuffle(seed=42).train_test_split(test_size=0.1)

# 3. Rename for clarity
train_dataset = split["train"]
val_dataset   = split["test"]

print(train_dataset)  
print(val_dataset)    # ~10% of rows


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['anchor', 'positive'],
    num_rows: 2449
})
Dataset({
    features: ['anchor', 'positive'],
    num_rows: 273
})


In [359]:
loss = losses.MultipleNegativesSymmetricRankingLoss(model=model)

In [365]:
model = SentenceTransformer("microsoft/mpnet-base")

No sentence-transformers model found with name microsoft/mpnet-base. Creating a new one with mean pooling.
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [379]:
args = SentenceTransformerTrainingArguments(
    output_dir="./models/microsoft_mpnet-base",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    report_to="none",
    logging_steps=100,
    save_steps=600,
)

In [380]:
trainer = SentenceTransformerTrainer(
    args=args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    loss=loss,
)
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
100,0.599600
200,0.500300
300,0.483900
400,0.426800
500,0.480100
600,0.493700
700,0.515000


TrainOutput(global_step=770, training_loss=0.5115868803742644, metrics={'train_runtime': 95.6248, 'train_samples_per_second': 128.053, 'train_steps_per_second': 8.052, 'total_flos': 0.0, 'train_loss': 0.5115868803742644, 'epoch': 5.0})

In [381]:
all_cats = pd.read_csv("categories.csv", header=None)[0].tolist()

In [383]:
from sentence_transformers import util

bi_encoder = SentenceTransformer("./models/microsoft_mpnet-base/checkpoint-770")
titles = test["Title"].unique().tolist()
title_embs    = bi_encoder.encode(titles,    convert_to_tensor=True)
category_embs = bi_encoder.encode(all_cats,  convert_to_tensor=True)


In [386]:
len(title_embs)

199

In [387]:
len(category_embs)

582

In [389]:
logits = title_embs @ category_embs.T
most_relevant_categories = torch.topk(logits, k = 5).indices

In [405]:
for i in range(len(most_relevant_categories)):
    logit = most_relevant_categories[i]
    # print(most_relevant_categories)
    print('---')
    print(titles[i])
    print('Categories:')
    for cat_id in logit:
        print(all_cats[cat_id])

---
What It Takes: Lessons in the Pursuit of Excellence by Stephen A. Schwarzman
Categories:
Books on Success
Books on Management
Books on Goal Setting
Books on Focus
Growth Mindset Books
---
The Practice: Shipping Creative Work by Seth Godin
Categories:
Books on Creativity
Books on Design Thinking
Books on Content Creation
Books on Productivity
Books on Writing
---
The Ideal Team Player: How to Recognize and Cultivate the Three Essential Virtues by Patrick Lencioni
Categories:
Books on Team Building
Books on Confidence
Books on Sports Psychology
Books on Success
Books on Plato
---
4 Essential Keys to Effective Communication in Love, Life, Work — Anywhere! by Bento C. Leal III
Categories:
Books on Communication
Books on Relationships
Books on Conversation
Books on Marriage
Books on Negotiation
---
The Undocumented Americans by Karla Cornejo Villavicencio
Categories:
Books about Immigration
Books on Mexican History
Books about Mexico
Books on Civil Rights
Books on Black History
---
Emot

In [408]:

def precision_at_k(logits, k = 5):
    most_relevant_categories = torch.topk(logits, k = 5).indices
    for i in range(len(most_relevant_categories)):
        cat_list_ground_truth = df.iloc[i]['CategoryList']
        for cat_id in logit:
            print(all_cats[cat_id])
            if all_cats[cat_id] in cat_list_ground_truth
    
def recall_at_k(logits, k = 1):
    pass

def mean_average_precision()

In [409]:
def precision_at_k_simple(logits, ground_truth, k=5):
    """
    logits:   torch.Tensor [N, C]
    ground_truth: list of length N, each a list (or set) of true category indices
    """
    topk = torch.topk(logits, k, dim=1).indices.tolist()  # [[i1,i2,…], …]
    precisions = []
    for preds, true_labels in zip(topk, ground_truth):
        true_set = set(true_labels)
        # pick only those preds that appear in the full ground-truth
        n_correct = sum((1 for p in preds if p in true_set))
        precisions.append(n_correct / k)
    return sum(precisions) / len(precisions)


In [425]:
import pandas as pd
import numpy as np
from sentence_transformers import util

# --- 1) Prepare your DataFrame ---
# df must already exist, with columns "Title" (str) and "Categories" (newline-separated str)
df['CategoryList'] =  (
    df['Categories']
      .fillna('')           
      .astype(str)           
      .str.split('\n')      
)

all_cats = sorted({cat for cats in df['CategoryList'] for cat in cats})

book_emb = model.encode(df['Title'].tolist(), convert_to_numpy=True)
cat_emb  = model.encode(all_cats,            convert_to_numpy=True)


book_emb_norm = book_emb / np.linalg.norm(book_emb, axis=1, keepdims=True)
cat_emb_norm  = cat_emb  / np.linalg.norm(cat_emb,  axis=1, keepdims=True)
logits = book_emb_norm.dot(cat_emb_norm.T)  # shape [n_books, n_cats]

cat2idx = {cat:i for i,cat in enumerate(all_cats)}
ground_truth_idx = [
    [cat2idx[cat] for cat in cats] 
    for cats in df['CategoryList']
]

def precision_at_k(logits: np.ndarray, gt: list[list[int]], k: int) -> float:
    topk = np.argsort(-logits, axis=1)[:, :k]
    p = [ len(set(pred)&set(true))/k for pred,true in zip(topk, gt) ]
    return float(np.mean(p))

def recall_at_k(logits: np.ndarray, gt: list[list[int]], k: int) -> float:
    topk = np.argsort(-logits, axis=1)[:, :k]
    r = [ len(set(pred)&set(true))/(len(true) or 1) for pred,true in zip(topk, gt) ]
    return float(np.mean(r))

def mean_average_precision(logits: np.ndarray, gt: list[list[int]], k: int = None) -> float:
    sorted_inds = np.argsort(-logits, axis=1)
    APs = []
    for preds, true in zip(sorted_inds, gt):
        true_set = set(true)
        if not true_set:
            APs.append(1.0)
            continue
        if k is not None:
            preds = preds[:k]
        hits = 0; score = 0.0
        for rank,p in enumerate(preds, start=1):
            if p in true_set:
                hits += 1
                score += hits/rank
        APs.append(score/len(true_set))
    return float(np.mean(APs))

metrics = {
    'P@10':    precision_at_k(logits, ground_truth_idx, k=1),
    'R@15':    recall_at_k(logits,    ground_truth_idx, k=1),
    'mAP':     mean_average_precision(logits, ground_truth_idx),
    'mAP@15':  mean_average_precision(logits, ground_truth_idx, k=15),
}

for name, val in metrics.items():
    print(f"{name:6s}: {val:.4f}")


P@10  : 0.5234
R@15  : 0.4613
mAP   : 0.6386
mAP@15: 0.6210
